# Steam game review analysis

## 1. Imports

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import json
from tqdm import tqdm
import csv

debug = False
url = 'https://store.steampowered.com/search/?category1=998&supportedlang=english'

# URL contains # which the scraping loop will replace with the start value in each iteration
url_raw = 'https://store.steampowered.com/search/results/?query&start=#&count=50&dynamic_data=&sort_by=_ASC&category1=998&supportedlang=english&snr=1_7_7_230_7&infinite=1'
# url_raw = 'https://store.steampowered.com/search/results/?query&start=#&count=50&dynamic_data=&force_infinite=1&category1=998&supportedlang=english&filter=topsellers&ndl=1&snr=1_7_7_7000_7&infinite=1'

## 2. Get number of games on page

In [2]:
def scrape_num_games(url):
    page = requests.get(url) # Get game homepage
    
    soup = BeautifulSoup(page.content, "html.parser")
    div = soup.find("div", {"id": "search_results_filtered_warning"})
    
    children = [str(i) for i in div.children]
    result = [i for i in children if '<div>' in i][0] # First child div contains game count
    game_count = result[result.index(">")+1:result.index(" ")].replace(",", "") # Get number of games returned by query
    excluded_game_count = result[result.index(". ")+2:result.index(" titles")].replace(",", "") # Get number of games not shown due to search preferences
    total_count = int(game_count) - int(excluded_game_count) # Calculate total number of games returned and shown

    return total_count

game_count = scrape_num_games(url)
print(f'[INFO] {game_count} games found') 

[INFO] 79162 games found


## 3. Scrape game information

In [4]:
def scrape_game_info(records, genres, url):
    page = requests.get(url)
    
    result = str(page.content.decode())
    
    string = '<span class=\"title\">Steam Deck<\/span>'
    results = re.findall(r'<a href=\\"https:\\\/\\\/store\.steampowered\.com\\\/app.{1,2750}\\r\\n\\t\\t\\t\\t', result)

    if debug:
        iters = 1
    else:
        iters = len(results)
        
    for i in range(iters):
        current = results[i]

        # DEBUGGING - Print the info about the current game to see the request content structure
        if debug:
            print(current)

        try:
            # Get 'title'
            title = re.findall(r'class=\\"title\\">.{1,300}<\\\/span', current)[0]
            title = title[title.index(">")+1:title.index("<\/span>")]
    
            # Get 'prices'
            if re.search(r'class=\\"discount_prices\\">.{1,300}<\\\/div', current) == None: # No price information given
                discount_price = None
                final_price = None
            else:
                price = re.findall(r'class=\\"discount_prices\\">.{1,300}<\\\/div', current)[0] # All price information
                if "discount_original_price" in price: # The price of the game before discount
                    discount_price = price[price.index("original_price")+17:price.index("<\/div>")]
                    price = price[price.index("<\/div>")+7:]
                else: # There is no discount
                    discount_price = None
                if 'free' in price:
                    final_price = 'Free'
                else:
                    final_price = price[price.index('final_price\\')+14:price.index("<\/div>")] # The current price of the game, either regular or after discount
                if discount_price == None: # If no discount, set pre-discount price to same as current (final) price
                    discount_price = final_price

            # Get 'reviews'
            if re.search(r'class=\\"search_review_summary positive\\".{1,300}\\">', current) == None:
                review_class = None # Overwhelmingly positive, positive, mixed etc.
                reviews = None
                review_percent = None # Percent of reviews that are positive
                review_absolute = None # Number of reviews
            else:
                review = re.findall(r'class=\\"search_review_summary positive\\".{1,300}\\">', current)[0]
                review_class = review[review.index('html=\\"')+7:review.index("&lt")]
                reviews = review[review.index('&gt;')+4:review.index('.')]
                review_percent = reviews[:reviews.index('%')]
                review_absolute = reviews[reviews.index('the ')+4:reviews.index(' user')].replace(",", "")
                
            # Get 'date'
            if re.search(r'class=\\"col search_released responsive_secondrow\\">.{1,200}<\\\/div>', current) == None:
                date = None
            else:
                date = re.findall(r'class=\\"col search_released responsive_secondrow\\">.{1,200}<\\\/div>', current)[0]
                date = date[date.index(r'\r\n')+4:date.index('<\/div>')].lstrip().rstrip()
                if date == "": # If no date, will match to whitespace so will be stripped to empty string
                    date = None
    
            game_page_url = re.findall(r'<a href=\\"https:\\\/\\\/store\.steampowered\.com\\\/app.{1,100}\\r\\n\\t\\t\\t', current)[0]
            game_page_url = game_page_url[game_page_url.index("https"):game_page_url.index('\\"\\r')].replace("\\", "")
            # print(game_page_url)
            
            game_page = requests.get(game_page_url)
            soup = BeautifulSoup(game_page.content, "html.parser")
            genre_info = soup.find("div", {"id": "genresAndManufacturer"})
            # print(genres)
            game_genres = []
            children = genre_info.findChildren("a", recursive=True)
            for child in children:
                if "genre" in str(child):
                    # print(child.text)
                    game_genres.append(child.text)
            # print(game_genres)
            genres.append(game_genres)
            records.append([title, discount_price, final_price, review_percent, review_absolute, review_class, date])
            
           
        except:
            print('[INFO] Cannot parse following info.')
            print(current)
            print('=====')
            
    return records, genres

# Scrapes game info in batches of 50, so total number of games scraped = ~50*lim
if debug:
    lim = 1
else:
    lim = game_count // 50
records = []
genres = []
print('[INFO] Scraping... ')
for i in tqdm(range(lim)):
    records, genres = scrape_game_info(records, genres, url_raw.replace('#', str(i*50)))
    # if i % 5 == 0:
    #     with open('data_records_backup.csv', 'w', newline='') as f:
    #         writer = csv.writer(f)
    #         writer.writerows(records)
    #         f.close()
    #     with open('data_genres_backup.csv', 'w', newline='') as f:
    #         writer = csv.writer(f)
    #         writer.writerows(genres)
    #         f.close()

[INFO] Scraping... 


  2%|█▎                                                                        | 28/1583 [1:10:06<53:46:45, 124.51s/it]

[INFO] Cannot parse following info.
<a href=\"https:\/\/store.steampowered.com\/app\/605740\/Flashing_Lights__Police_Firefighting_Emergency_Services_EMS_Simulator\/?snr=1_7_7_230_150_29\"\r\n\t\t\t data-ds-appid=\"605740\" data-ds-itemkey=\"App_605740\" data-ds-tagids=\"[3968,1774,6378,4175,5363,3814,1644]\" data-ds-crtrids=\"[32976461,44213547]\" onmouseover=\"GameHover( this, event, 'global_hover', {&quot;type&quot;:&quot;app&quot;,&quot;id&quot;:605740,&quot;public&quot;:1,&quot;v6&quot;:1} );\" onmouseout=\"HideGameHover( this, event, 'global_hover' )\" class=\"search_result_row ds_collapse_flag \"\r\n\t\t   data-search-page=\"29\" data-gpnav=\"item\"\r\n\t\t   data-ds-steam-deck-compat-handled=\"true\">\r\n            <div class=\"col search_capsule\"><img src=\"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/605740\/capsule_sm_120.jpg?t=1702301411\" srcset=\"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/605740\/capsule_sm_120.jpg?t=1702301411 1x, https:\/\/cdn.clo

  2%|█▍                                                                         | 31/1583 [1:11:36<27:06:08, 62.87s/it]

[INFO] Cannot parse following info.
<a href=\"https:\/\/store.steampowered.com\/app\/2348100\/YEAH_YOU_WANT_THOSE_GAMES_RIGHT_SO_HERE_YOU_GO_NOW_LETS_SEE_YOU_CLEAR_THEM\/?snr=1_7_7_230_150_32\"\r\n\t\t\t data-ds-appid=\"2348100\" data-ds-itemkey=\"App_2348100\" data-ds-tagids=\"[597,1664,5537,4726,4136,4878,1719]\" data-ds-crtrids=\"[33518623]\" onmouseover=\"GameHover( this, event, 'global_hover', {&quot;type&quot;:&quot;app&quot;,&quot;id&quot;:2348100,&quot;public&quot;:1,&quot;v6&quot;:1} );\" onmouseout=\"HideGameHover( this, event, 'global_hover' )\" class=\"search_result_row ds_collapse_flag \"\r\n\t\t   data-search-page=\"32\" data-gpnav=\"item\"\r\n\t\t   data-ds-steam-deck-compat-handled=\"true\">\r\n            <div class=\"col search_capsule\"><img src=\"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/2348100\/capsule_sm_120.jpg?t=1689818514\" srcset=\"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/2348100\/capsule_sm_120.jpg?t=1689818514 1x, https:\/\/cdn.cl

...

 99%|██████████████████████████████████████████████████████████████████████████▏| 1565/1583 [22:08:00<11:49, 39.41s/it]

[INFO] Cannot parse following info.
<a href=\"https:\/\/store.steampowered.com\/app\/1460070\/Is_It_Wrong_To_Try_To_Rescue_Monster_Girls_From_The_Inquisition\/?snr=1_7_7_230_150_1566\"\r\n\t\t\t data-ds-appid=\"1460070\" data-ds-itemkey=\"App_1460070\" data-ds-tagids=\"[3799,4085,1664,5611,4136,4726,1742]\" data-ds-descids=\"[5]\" onmouseover=\"GameHover( this, event, 'global_hover', {&quot;type&quot;:&quot;app&quot;,&quot;id&quot;:1460070,&quot;public&quot;:1,&quot;v6&quot;:1} );\" onmouseout=\"HideGameHover( this, event, 'global_hover' )\" class=\"search_result_row ds_collapse_flag \"\r\n\t\t   data-search-page=\"1566\" data-gpnav=\"item\"\r\n\t\t   data-ds-steam-deck-compat-handled=\"true\">\r\n            <div class=\"col search_capsule\"><img src=\"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/1460070\/capsule_sm_120.jpg?t=1684826473\" srcset=\"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/1460070\/capsule_sm_120.jpg?t=1684826473 1x, https:\/\/cdn.cloudflare.stea

100%|██████████████████████████████████████████████████████████████████████████▊| 1578/1583 [22:15:41<02:48, 33.74s/it]

[INFO] Cannot parse following info.
<a href=\"https:\/\/store.steampowered.com\/app\/796660\/About_how_he_earned_a_suite_from_TFinancial_Platform_in_four_months\/?snr=1_7_7_230_150_1579\"\r\n\t\t\t data-ds-appid=\"796660\" data-ds-itemkey=\"App_796660\" data-ds-tagids=\"[113,597,599,492,9,122]\" data-ds-crtrids=\"[34173458]\" onmouseover=\"GameHover( this, event, 'global_hover', {&quot;type&quot;:&quot;app&quot;,&quot;id&quot;:796660,&quot;public&quot;:1,&quot;v6&quot;:1} );\" onmouseout=\"HideGameHover( this, event, 'global_hover' )\" class=\"search_result_row ds_collapse_flag \"\r\n\t\t   data-search-page=\"1579\" data-gpnav=\"item\"\r\n\t\t   data-ds-steam-deck-compat-handled=\"true\">\r\n            <div class=\"col search_capsule\"><img src=\"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/796660\/capsule_sm_120.jpg?t=1582483362\" srcset=\"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/796660\/capsule_sm_120.jpg?t=1582483362 1x, https:\/\/cdn.cloudflare.steamstatic.

100%|███████████████████████████████████████████████████████████████████████████| 1583/1583 [22:18:41<00:00, 50.74s/it]


### 4. Save game info to CSV

In [7]:
with open('data_records_backup.csv', 'w', newline='', encoding='utf8') as f:
    writer = csv.writer(f)
    writer.writerows(records)
    f.close()
with open('data_genres_backup.csv', 'w', newline='', encoding='utf8') as f:
    writer = csv.writer(f)
    writer.writerows(genres)
    f.close()

### (Optional) Load game info from CSV

In [23]:
records = []
with open('data_records_backup.csv', encoding='utf8') as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        records.append(row)

print(len(records))

records_df = pd.DataFrame(records, columns=['title', 'pre-discount-price', 'sale-price', 'positive-review-percent', 'num-reviews', 'review-class', 'date-released'])
print(records_df)
# genres = []
# with open('data_genres_backup.csv', encoding='utf8') as csvfile:
#     reader = csv.reader(csvfile) # change contents to floats
#     for row in reader: # each row is a list
#         genres.append(row)

76567
                           title pre-discount-price sale-price  \
0                Baldur's Gate 3             £49.99     £44.99   
1                 Lethal Company              £8.50      £8.50   
2                 Cyberpunk 2077                                 
3               Counter-Strike 2               Free       Free   
4                  Call of Duty®                                 
...                          ...                ...        ...   
76562                  Seek Love             £10.99      £1.09   
76563  Age of Cards - Ra's Chess                                 
76564                  Thrunt XL                                 
76565                       荒野寻踪              £8.50      £8.50   
76566                Unsacrifice                                 

      positive-review-percent num-reviews             review-class  \
0                          96      454247  Overwhelmingly Positive   
1                          98      166011  Overwhelmingly Pos

### 5. One-hot encode genres and convert data to pandas DF

In [24]:
def one_hot_encode_genres(X):

    # Create array of all genres in X
    genres = []
    for i in X:
        for j in i:
            if j not in genres:
                genres.append(j)
    
    # print(genres)

    # Initialise empty genre dictionary
    genre_dict = {}
    for i in genres:
        genre_dict[i] = []
    
    # print(genre_dict)

    # Add 1 for each genre in game i, add 0 for each genre not in game i
    for i in X:
        for j in i:
            genre_dict[j].append(1)
        for k in [x for x in genres if x not in i]:
            genre_dict[k].append(0)

    return genre_dict

# one_hot_genres = one_hot_encode_genres(genres)
# genre_df = pd.DataFrame.from_dict(one_hot_genres)

df = pd.DataFrame(records, columns=['title', 'pre-discount-price', 'sale-price', 'positive-review-percent', 'num-reviews', 'review-class', 'date-released'])
df = pd.concat([df, genre_df], axis=1)
df

,title,pre-discount-price,sale-price,positive-review-percent,num-reviews,review-class,date-released,Adventure,RPG,Strategy,...,Movie,Video Production,Software Training,Accounting,Photo Editing,Documentary,Episodic,Short,Tutorial,360 Video
0,Baldur's Gate 3,£49.99,£44.99,96,454247,Overwhelmingly Positive,"3 Aug, 2023",1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Lethal Company,£8.50,£8.50,98,166011,Overwhelmingly Positive,"23 Oct, 2023",1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cyberpunk 2077,,,,,,"9 Dec, 2020",0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Counter-Strike 2,Free,Free,87,7802681,Very Positive,"21 Aug, 2012",0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Call of Duty®,,,,,,"27 Oct, 2022",0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76562,Seek Love,£10.99,£1.09,,,,"21 Nov, 2018",1,0,1,...,0,0,0,0,0,0,0,0,0,0
76563,Age of Cards - Ra's Chess,,,,,,Coming soon,0,0,1,...,0,0,0,0,0,0,0,0,0,0
76564,Thrunt XL,,,,,,Coming soon,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76565,荒野寻踪,£8.50,£8.50,,,,"1 Jun, 2018",1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df = df_raw
df

,title,pre-discount-price,sale-price,positive-review-percent,num-reviews,review-class,date-released,Adventure,RPG,Strategy,...,Movie,Video Production,Software Training,Accounting,Photo Editing,Documentary,Episodic,Short,Tutorial,360 Video
0,Baldur's Gate 3,£49.99,£44.99,96,454247,Overwhelmingly Positive,None,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Lethal Company,£8.50,£8.50,98,166011,Overwhelmingly Positive,None,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cyberpunk 2077,None,None,None,None,None,None,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Counter-Strike 2,Free,Free,87,7802681,Very Positive,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Call of Duty®,None,None,None,None,None,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76562,Seek Love,£10.99,£1.09,None,None,None,None,1,0,1,...,0,0,0,0,0,0,0,0,0,0
76563,Age of Cards - Ra's Chess,None,None,None,None,None,None,0,0,1,...,0,0,0,0,0,0,0,0,0,0
76564,Thrunt XL,None,None,None,None,None,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76565,荒野寻踪,£8.50,£8.50,None,None,None,None,1,0,1,...,0,0,0,0,0,0,0,0,0,0


### 6. Change date format

In [41]:
from sklearn.preprocessing import LabelEncoder

# Change month into number format
def switch(date):
    if date == 'Jan':
        return '01'
    elif date == 'Feb':
        return '02'
    elif date == 'Mar':
        return '03'
    elif date == 'Apr':
        return '04'
    elif date == 'May':
        return '05'
    elif date == 'Jun':
        return '06'
    elif date == 'Jul':
        return '07'
    elif date == 'Aug':
        return '08'
    elif date == 'Sep':
        return '09'
    elif date == 'Oct':
        return '10'
    elif date == 'Nov':
        return '11'
    elif date == 'Dec':
        return '12'
    return None

df_raw = df

# Change date format to YYYY-MM-DD
print('[INFO] Changing date format...')
converted_dates = []
for i in tqdm(range(len(df))):
    date = str(df['date-released'].iloc[i])
    if len(date.split(" ")) == 3 and "," in date:
        try:
            year = date[date.index(", ")+2:]
            month = date[date.index(" ")+1:date.index(",")]
            day = date[:date.index(" ")]
            month = switch(month) # Turn month into number format
            date = year + '-' + month + '-' + day.zfill(2)
        except:
            date = 'None'
    elif len(date.split("-")) == 3:
        date = date
    else:
        date = 'None'
    converted_dates.append(date)

# print(converted_dates)
df['date-released'] = converted_dates
df

[INFO] Changing date format...


100%|████████████████████████████████████████████████████████████████████████| 76567/76567 [00:00<00:00, 135393.96it/s]


,title,pre-discount-price,sale-price,positive-review-percent,num-reviews,review-class,date-released,Adventure,RPG,Strategy,...,Movie,Video Production,Software Training,Accounting,Photo Editing,Documentary,Episodic,Short,Tutorial,360 Video
0,Baldur's Gate 3,£49.99,£44.99,96,454247,Overwhelmingly Positive,2023-08-03,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Lethal Company,£8.50,£8.50,98,166011,Overwhelmingly Positive,2023-10-23,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cyberpunk 2077,,,,,,2020-12-09,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Counter-Strike 2,Free,Free,87,7802681,Very Positive,2012-08-21,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Call of Duty®,,,,,,2022-10-27,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76562,Seek Love,£10.99,£1.09,,,,2018-11-21,1,0,1,...,0,0,0,0,0,0,0,0,0,0
76563,Age of Cards - Ra's Chess,,,,,,None,0,0,1,...,0,0,0,0,0,0,0,0,0,0
76564,Thrunt XL,,,,,,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76565,荒野寻踪,£8.50,£8.50,,,,2018-06-01,1,0,1,...,0,0,0,0,0,0,0,0,0,0


### 7. Clean up data - remove missing values and change formatting

In [54]:
# Remove rows where review-class is None
df = df.loc[df['review-class'].notna()]

df = df.loc[df['sale-price'].notna()]
df = df.loc[df['pre-discount-price'].notna()]

converted_disc_prices = []
converted_sale_prices = []
review_classes = []
for i in tqdm(range(len(df))):
    # Change empty review-class values to None
    if len(df['review-class'].iloc[i]) == 0:
        review_class = None
    else:
        review_class = df['review-class'].iloc[i]
    
    discount_price = df['pre-discount-price'].iloc[i]
    sale_price = df['sale-price'].iloc[i]

    if sale_price == 'Free':
        sale_price = '0.00'
        discount_price = '0.00'
    if len(sale_price) == 0:
        sale_price = None
    else:
        sale_price = sale_price.replace("£", "")
    if len(discount_price) == 0:
        discount_price = None
    else:
        discount_price = discount_price.replace("£", "")
    

    converted_disc_prices.append(discount_price)
    converted_sale_prices.append(sale_price)
    review_classes.append(review_class)
    
df['pre-discount-price'] = converted_disc_prices
df['sale-price'] = converted_sale_prices
df['review-class'] = review_classes

df = df.loc[df['sale-price'].notna()]
df = df.loc[df['pre-discount-price'].notna()]

df = df.loc[df['review-class'].notna()]

df

100%|█████████████████████████████████████████████████████████████████████████| 65384/65384 [00:01<00:00, 43044.40it/s]


,title,pre-discount-price,sale-price,positive-review-percent,num-reviews,review-class,date-released,Adventure,RPG,Strategy,...,Movie,Video Production,Software Training,Accounting,Photo Editing,Documentary,Episodic,Short,Tutorial,360 Video
0,Baldur's Gate 3,49.99,44.99,96,454247,Overwhelmingly Positive,2023-08-03,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Lethal Company,8.50,8.50,98,166011,Overwhelmingly Positive,2023-10-23,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Counter-Strike 2,0.00,0.00,87,7802681,Very Positive,2012-08-21,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Red Dead Redemption 2,59.99,19.79,91,464773,Very Positive,2019-12-05,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Hogwarts Legacy,49.99,24.99,92,175314,Very Positive,2023-02-10,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33621,Orake Classic,0.00,0.00,71,639,Mostly Positive,2017-06-16,1,1,1,...,0,0,0,0,0,0,0,0,0,0
33622,PooShooter: Toilet Invaders,0.00,0.00,72,93,Mostly Positive,2016-10-03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33623,Gauntlet of IRE,0.00,0.00,74,181,Mostly Positive,2017-12-15,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33624,Iron League,0.00,0.00,72,804,Mostly Positive,2018-03-14,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### 8. Save DataFrame to CSV

In [57]:
def save_df(df):
    df.to_csv('data.csv', header=True, index=False)

save_df(df)
print('[INFO] Dataframe saved.')

[INFO] Dataframe saved.
